In [ ]:
import os
import pycocotools
os.getcwd()


In [ ]:
%%bash
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0
cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

In [ ]:
import numpy as np
import torch
import torch.utils.data
from PIL import Image
from PIL import ImageDraw
import pandas as pd
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from engine import train_one_epoch, evaluate
import utils
import transforms as T
import torchvision
import xml.etree.ElementTree as ET
import glob

In [ ]:
def extract_BBoxes(filename):
   
    root = ET.parse(filename).getroot()
    
    boxes = list()
    names = list()
    for objct in root.findall(".//object"):
        name = objct.find('name').text
        xmin = int(objct.find('bndbox/xmin').text)
        ymin = int(objct.find('bndbox/ymin').text)
        xmax = int(objct.find('bndbox/xmax').text)
        ymax = int(objct.find('bndbox/ymax').text)
        names.append(name)
        boxes.append([xmin,ymin,xmax,ymax])

    return [boxes,names]
    
def load_dataset(path, deleteFiles = False):
    files_xml = [f for f in glob.glob(path + "/*.xml")] # comes in random order

    
    imgbbox = dict()
    print(len(files_xml))
    for file in files_xml:  
        
        imgFilePath = file[:-3] + "jpg"
        if os.path.exists(imgFilePath):  
            lbl_bbox = extract_BBoxes(file)  # Gets the bbox information
            
            #print(lbl_bbox[0])
            
            imgbbox.update({imgFilePath.replace(path+"/",''): lbl_bbox})
            
        elif deleteFiles:
            print("Found xml with no jpg")
            print("Deleting xml file: %s" %file)
            os.remove(file)
            print("Deleted")

    return imgbbox

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
pathDataset = 'drive/MyDrive/Project2'
imgbbox = load_dataset(pathDataset, deleteFiles=False)
imgbbox['2 (20).jpg']


Obtaining the data

Class containing all data for training

In [ ]:
class GunDataset(torch.utils.data.Dataset):
    
    def __init__(self, dicPics,categories, path, transforms=None): 
        self.path = path
        self.dicPics = dicPics
        self.transforms = transforms
        self.categories = categories
        self.imgs = [o for o in dicPics]

    def __getitem__(self, idx):
        img = Image.open(self.path +"/"+ self.imgs[idx]).convert("RGB")        
        box_list = self.dicPics[self.imgs[idx]][0]
        
        boxes = torch.as_tensor(box_list, dtype=torch.float32)
        num_objs = len(box_list)
        labels_list =  self.dicPics[self.imgs[idx]][1]

        # there is only one class
        #labels = torch.ones((num_objs,), dtype=torch.int64) 

        # multible classes
        labels = torch.zeros((num_objs,), dtype=torch.int64)

        for i in range(num_objs):
          labels[i] = self.categories[labels_list[i]]
        
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:,0])

        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        
        if self.transforms is not None:
            img, target = self.transforms(img, target)
        
        return img, target
    
    def __len__(self):
        return len(self.imgs)

Checks if the class is correct and returns the expected values

In [ ]:
cat = {'handgun': 1,'rifle': 2}
dataset = GunDataset(dicPics = imgbbox,categories = cat, path = pathDataset, transforms = None) #, categories = cat
dataset.__getitem__(10)

In [ ]:
image = Image.open(pathDataset + '/1 (107).jpg')
image

Downloads and configures the model for our dataset

In [ ]:
def get_model(num_classes):
  # load an object detection model pre-trained on COCO
  model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
  # get the number of input features for the classifier
  in_features = model.roi_heads.box_predictor.cls_score.in_features
  # replace the pre-trained head with a new on
  model.roi_heads.box_predictor = FastRCNNPredictor(in_features,num_classes)
   
  return model

In [ ]:
def get_transform(train):
    transforms = []
   # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
      # during training, randomly flip the training images
      # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [ ]:
# use our dataset and defined transformations
dataset = GunDataset(dicPics=imgbbox, categories = cat, path = pathDataset, transforms = get_transform(train=True))      # Training
dataset_test = GunDataset(dicPics = imgbbox, categories = cat, path = pathDataset, transforms = get_transform(train=False)) # Testing

# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-40])  
#dataset = torch.utils.data.Subset(dataset, indices[:100])  # testing

dataset_test = torch.utils.data.Subset(dataset_test, indices[-40:])
#dataset_test = torch.utils.data.Subset(dataset_test, indices[-30:])


# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
              dataset, batch_size=8, shuffle=True, num_workers=4,
              collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
         dataset_test, batch_size=1, shuffle=False, num_workers=4,
         collate_fn=utils.collate_fn)

print("We have: {} examples, {} are training and {} testing".format(len(indices), len(dataset), len(dataset_test)))

In [ ]:
torch.cuda.is_available()

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# our dataset has two classes only - gun and not gun
num_classes = 3
# get the model using our helper function
model = get_model(num_classes)
# move model to the right device
model.to(device)
# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by # 10x every 3 epochs

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

## TRAIN NEW MODEL

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
   # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch+1, print_freq=10)
   # update the learning rate

    lr_scheduler.step()
   # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

In [ ]:
 # Saves model to folder trainedModels
    
path_trnd_model = "drive/MyDrive/trainedModels"
if os.path.isdir(path_trnd_model) is False:
    os.mkdir(path_trnd_model)

state = {'epoch': num_epochs +1, 'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict() }
torch.save(state, path_trnd_model+"/model_3Classes_15epochs")

#torch.save(model.state_dict(), )

## TRAIN OLD MODEL

In [ ]:
def load_checkpoint(model, optimizer=None, filename=None):
    # Note: Input model & optimizer should be pre-defined.  This routine only updates their states.
    start_epoch = 0
    if os.path.isfile(filename):
        print("=> loading checkpoint '{}'".format(filename))
        checkpoint = torch.load(filename)
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        if optimizer is not None:
          optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})".format(filename, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(filename))

    return model, optimizer, start_epoch

In [ ]:
path_trnd_model = "drive/MyDrive/trainedModels"
model, optimizer, start_epoch = load_checkpoint(get_model(num_classes = 3), optimizer, filename=path_trnd_model+"/model_3Classes_15epochs")
model = model.to(device)

# individually transfer the optimizer parts...
for state in optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.to(device)

In [ ]:
# let's train it for 0 epochs
num_epochs = 10
for epoch in range(num_epochs):
   # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch+1, print_freq=10)
# update the learning rate

    lr_scheduler.step()
   # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

In [ ]:
state = {'epoch': num_epochs +1, 'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict() }
torch.save(state, path_trnd_model+"/model_3Classes_15epochs")


In [ ]:
# Saves the final model, not trainable anymore
#torch.save(model.state_dict(), path_trnd_model+"/model_3Classes_15epochs_finished")

# PREDICT 

In [ ]:
# To load trained model
model_name = "model_3Classes_20epochs_finished"

loaded_model = get_model(num_classes = 3)

if os.path.isfile(path_trnd_model +"/"+ model_name):
    loaded_model.load_state_dict(torch.load( path_trnd_model +"/"+ model_name))
else:
    print("Wrong path or filename")

In [ ]:
loaded_model = model

device = torch.device("cuda")
#loaded_model.to(device)
cat_color = {1:"yellow", 2:"orange"}

for idx in range(len(dataset_test)):
    img, _ = dataset_test[idx]
    label_boxes = np.array(dataset_test[idx][1]["boxes"])
    #put the model in evaluation mode
    loaded_model.eval()
    with torch.no_grad():
        prediction = loaded_model([img.to(device)])

    image = Image.fromarray(img.mul(255).permute(1, 2,0).byte().numpy())
    draw = ImageDraw.Draw(image)

    cat_rev = {cat[o]: o for o in cat}
    # draw groundtruth
    for elem in range(len(label_boxes)):
        draw.rectangle([(label_boxes[elem][0], label_boxes[elem][1]),
        (label_boxes[elem][2], label_boxes[elem][3])], 
        outline ="green", width =3)
        
    for element in range(len(prediction[0]["boxes"])):

        boxes = prediction[0]["boxes"][element].cpu().numpy()
        score = np.round(prediction[0]["scores"][element].cpu().numpy(),
                        decimals= 4)
        if score > 0.8:
            draw.rectangle([(boxes[0], boxes[1]), (boxes[2], boxes[3])], outline =cat_color[np.int(prediction[0]["labels"][element])], width =3)
            draw.text((boxes[0], boxes[1]), text = str(score)+ " " + cat_rev[np.int(prediction[0]["labels"][element])] , fill="#000")
    display(image)


In [ ]:
pred_list_labels = []
gt_list_labels = []


for idx in range(len(dataset_test)):

    gt_lab = np.array([])
    img, targets = dataset_test[idx]
    
    if len(targets) is not 0:
        gt_lab = targets["labels"].numpy()
    gt_list_labels.append(gt_lab)
        
    with torch.no_grad():
        prediction = loaded_model([img.to(device)])[0]


    pred_list_labels.append(prediction["labels"].cpu().numpy()[prediction["scores"].cpu().numpy() > 0.8])
    
    if (idx+1) % 5 == 0:
        print(idx+1, "/", len(dataset_test))

In [ ]:
labels = {cat[o]: o for o in cat}

TP, FP, FN, TN = 0, 0, 0, 0

for idx in range(len(dataset_test)):
    gt_labels = gt_list_labels[idx]
    pred_labels = pred_list_labels[idx]
    
    if  (gt_labels.any()  == True  and pred_labels.any() == True):
        TP += 1
    elif (gt_labels.any() == False and pred_labels.any() == True):
        FP += 1
    elif (gt_labels.any() == True  and pred_labels.any() == False):
        FN += 1
    elif (gt_labels.any() == False and pred_labels.any() == False):
        TN += 1
    

print("\nTP: {}, FP: {}, FN: {}, TN: {}  |  Accuracy: {:.4f},  Precision: {:.3f},  Recall: {:.3f}".format(
                                                       TP,FP,FN,TN,((TP+TN)/(FP+FN+TP+TN)), TP/(TP+FP), TP/(TP+FN) )) 

In [ ]:
labels = {cat[o]: o for o in cat}
tp, tn, fp, fn = np.zeros(len(labels)), np.zeros(len(labels)), np.zeros(len(labels)), np.zeros(len(labels))

for idx in range(1,len(dataset_test)):
    gt_labels = gt_list_labels[idx]
    pred_labels = pred_list_labels[idx]
    
    count_gt, count_pred = np.zeros(len(labels)), np.zeros(len(labels))
    
    for label in gt_labels:
        count_gt[label-1] += 1
    for label in pred_labels:
        count_pred[label-1] += 1
        
    #print("\t\t", count_gt,"\t", count_pred, "\n")
    

    
    for i in range(len(labels)):
        difference = count_pred[i] - count_gt[i]
        
        if difference == 0:
            tp[i] += count_pred[i]
            if count_pred[i] == 0 and count_gt[i] == 0:
                tn[i] += 1
        elif difference > 0:
            tp[i] += count_gt[i]
            fp[i] += difference

        else:
            tp[i] += count_pred[i]
            fn[i] -= difference

# print statistics
for i in range(len(labels)):
    print()
    print("{}\tTP: {}, FP: {}, FN: {};\t   |  Precision: {:.3f},  Recall: {:.3f}".format(
        labels[i+1] + "   ", tp[i] ,fp[i], fn[i], tp[i]/(tp[i]+fp[i]), tp[i]/(tp[i]+fn[i]) ))